In [1]:
# Code aus dem vorigen Schritt zur Wiederverwendung
import pandas as pd
import numpy as np # Wichtig für die zweite Alternative

daten = {
    'Name': ['Anna', 'Mark', 'Julia', 'Tom'],
    'Studiengang': ['Informatik', 'BWL', 'Maschinenbau', 'Informatik'],
    'Semester': [4, 6, 2, 4],
    'Note': [1.7, 2.3, 1.0, 2.7]
}
df = pd.DataFrame(daten)

___
## Binning
Alternative 1: `pd.cut()`` (Die ideale Lösung für Binning)
Die Funktion `pd.cut()`` ist speziell dafür gemacht, numerische Daten in "Bins" (Kategorien oder Intervalle) einzuteilen. Das ist genau das, was wir hier tun. Diese Methode ist extrem schnell und der empfohlene Weg für solche Aufgaben

In [3]:
# Schritt 1: Definiere die Grenzen der Intervalle (Bins).
# Die Grenzen sind: (0 -> 1.5], (1.5 -> 2.5], (2.5 -> 3.5], (3.5 -> 5.0]
# Die Note 1.0 ist der beste Wert, 5.0 der schlechteste.
bins = [0, 1.5, 2.5, 3.5, 5.0]

# Schritt 2: Definiere die Labels für jedes Intervall.
labels = ['Sehr Gut', 'Gut', 'Befriedigend', 'Ausreichend']

# Schritt 3: Wende pd.cut() auf die 'Note'-Spalte an.
# `right=True` (Standard) bedeutet, dass die rechte Grenze zum Intervall gehört (z.B. 1.5 zählt zu 'Sehr Gut').
df['Praedikat_cut'] = pd.cut(df['Note'], bins=bins, labels=labels, right=True)

print("--- Effiziente Kategorisierung mit pd.cut() ---")
print(df)

--- Effiziente Kategorisierung mit pd.cut() ---
    Name   Studiengang  Semester  Note Praedikat_cut
0   Anna    Informatik         4   1.7           Gut
1   Mark           BWL         6   2.3           Gut
2  Julia  Maschinenbau         2   1.0      Sehr Gut
3    Tom    Informatik         4   2.7  Befriedigend


___

## flexible multiple Bedingungen
Alternative 2: np.select() (Die flexible Lösung für multiple Bedingungen)
np.select() aus der NumPy-Bibliothek ist eine weitere, sehr schnelle und vektorisierte Möglichkeit, komplexe "Wenn-Dann"-Logik umzusetzen. Man erstellt eine Liste von Bedingungen und eine Liste der entsprechenden Werte.

In [5]:
# Schritt 1: Definiere eine Liste der Bedingungen.
# Die Reihenfolge ist wichtig, da die erste erfüllte Bedingung "gewinnt".
conditions = [
    df['Note'] <= 1.5,
    df['Note'] <= 2.5,
    df['Note'] <= 3.5,
    df['Note'] > 3.5
]

# Schritt 2: Definiere eine Liste der dazugehörigen Werte (Choices).
choices = ['Sehr Gut', 'Gut', 'Befriedigend', 'Ausreichend']

# Schritt 3: Wende np.select an.
# `default` wird verwendet, falls keine Bedingung zutrifft.
df['Praedikat_select'] = np.select(conditions, choices, default='Unbekannt')

print("--- Effiziente Kategorisierung mit np.select() ---")
print(df)

--- Effiziente Kategorisierung mit np.select() ---
    Name   Studiengang  Semester  Note Praedikat_cut Praedikat_select
0   Anna    Informatik         4   1.7           Gut              Gut
1   Mark           BWL         6   2.3           Gut              Gut
2  Julia  Maschinenbau         2   1.0      Sehr Gut         Sehr Gut
3    Tom    Informatik         4   2.7  Befriedigend     Befriedigend


___
## String operationen

**Szenario**: Du möchtest eine Textoperation auf jede Zelle einer Spalte anwenden (z.B. alles in Kleinbuchstaben umwandeln, Textteile ersetzen oder prüfen, ob ein Wort enthalten ist).

**Ineffizienter Weg mit .apply()**:
```python
df['Studiengang'].apply(lambda x: x.lower())
```

**Effizienter Weg**: Pandas bietet für Spalten mit dem Datentyp object (meistens Strings) den .str-Accessor an. Dieser führt die String-Operationen optimiert und vektorisiert auf der gesamten Spalte aus.

In [8]:
import pandas as pd

df = pd.DataFrame({
    'Name': ['Anna', 'Mark', 'Julia'],
    'Studiengang': ['INFO-Master', 'BWL-Bachelor', 'MSc Maschinenbau']
})

# ALLES IN KLEINBUCHSTABEN UMWANDELN
df['Studiengang_klein'] = df['Studiengang'].str.lower()

# PRÜFEN, OB EIN STRING ENTHALTEN IST (ERGIBT BOOLEAN)
df['Ist_Master'] = df['Studiengang'].str.contains('Master|MSc', case=False)

# TEXT ERSETZEN
df['Studiengang_sauber'] = df['Studiengang'].str.replace('-', ' ')

print("--- Manipulation mit dem .str-Accessor ---")
print(df)

print(df.dtypes)

--- Manipulation mit dem .str-Accessor ---
    Name       Studiengang Studiengang_klein  Ist_Master Studiengang_sauber
0   Anna       INFO-Master       info-master        True        INFO Master
1   Mark      BWL-Bachelor      bwl-bachelor       False       BWL Bachelor
2  Julia  MSc Maschinenbau  msc maschinenbau        True   MSc Maschinenbau
Name                  object
Studiengang           object
Studiengang_klein     object
Ist_Master              bool
Studiengang_sauber    object
dtype: object


___
**Szenario**: Du hast eine Spalte mit Datums- oder Zeitwerten und möchtest Teile davon extrahieren (z.B. das Jahr, den Monat, den Wochentag).

**Ineffizienter Weg mit .apply()**:
```python
df['Datum'].apply(lambda x: x.year)
```

**Effizienter Weg**: Ähnlich wie .str gibt es für Spalten mit Datums-Objekten (Datentyp datetime64) den .dt-Accessor.



In [16]:
import pandas as pd

df_termine = pd.DataFrame({
    'Ereignis': ['Klausur A', 'Projekt-Abgabe', 'Semesterstart'],
    # Wichtig: Zuerst in ein echtes Datumsformat umwandeln
    'Datum': pd.to_datetime(['2025-07-22', '2025-08-15', '2025-10-01'])
})

# JAHR, MONAT UND WOCHENTAG EXTRAHIEREN
df_termine['Jahr'] = df_termine['Datum'].dt.year
df_termine['Monat'] = df_termine['Datum'].dt.month
df_termine['Wochentag'] = df_termine['Datum'].dt.day_name()
df_termine['dow'] = df_termine["Datum"].dt.dayofweek 

print("--- Manipulation mit dem .dt-Accessor ---")
print(df_termine)
print("\nDatentyp der 'Datum'-Spalte:", df_termine['Datum'].dtype)

--- Manipulation mit dem .dt-Accessor ---
         Ereignis      Datum  Jahr  Monat  Wochentag  dow
0       Klausur A 2025-07-22  2025      7    Tuesday    1
1  Projekt-Abgabe 2025-08-15  2025      8     Friday    4
2   Semesterstart 2025-10-01  2025     10  Wednesday    2

Datentyp der 'Datum'-Spalte: datetime64[ns]


___
**Szenario**: Du möchtest eine neue Spalte basierend auf einer einzigen "Wenn-Dann-Sonst"-Bedingung erstellen.

**Ineffizienter Weg mit .apply()**:
```python
df['Note'].apply(lambda x: 'Bestanden' if x <= 4.0 else 'Durchgefallen')
```

**Effizienter Weg**: Für eine einzelne Bedingung ist np.where von NumPy die schnellste und lesbarste Wahl. Die Syntax ist np.where(bedingung, wert_wenn_wahr, wert_wenn_falsch).

In [10]:
import numpy as np
df_noten = pd.DataFrame({'Note': [1.3, 2.7, 5.0, 3.0, 4.0]})

# STATUS BASIEREND AUF NOTE SETZEN
df_noten['Status'] = np.where(df_noten['Note'] <= 4.0, 'Bestanden', 'Durchgefallen')

print("--- Effiziente Wenn-Dann-Logik mit np.where() ---")
print(df_noten)

--- Effiziente Wenn-Dann-Logik mit np.where() ---
   Note         Status
0   1.3      Bestanden
1   2.7      Bestanden
2   5.0  Durchgefallen
3   3.0      Bestanden
4   4.0      Bestanden


___
**Szenario**: Du möchtest Werte in einer Spalte basierend auf einer festen Zuordnung (einem Dictionary) ersetzen.

**Ineffizienter Weg mit .apply()**:
```python
df['Kuerzel'].apply(lambda x: mapping_dict.get(x))
```

**Effizienter Weg**: Die .map()-Methode einer Series ist genau für diesen Zweck optimiert. Sie ist extrem schnell für solche "Lookups".



In [11]:
df_kuerzel = pd.DataFrame({'Studiengang_Kuerzel': ['INF', 'BWL', 'MB', 'INF']})

# ZUORDNUNG (MAPPING) DEFINIEREN
studiengang_map = {
    'INF': 'Informatik',
    'BWL': 'Betriebswirtschaftslehre',
    'MB': 'Maschinenbau'
}

# KUERZEL DURCH VOLLSTÄNDIGE NAMEN ERSETZEN
df_kuerzel['Studiengang_Voll'] = df_kuerzel['Studiengang_Kuerzel'].map(studiengang_map)

print("--- Effizientes Ersetzen mit .map() ---")
print(df_kuerzel)

--- Effizientes Ersetzen mit .map() ---
  Studiengang_Kuerzel          Studiengang_Voll
0                 INF                Informatik
1                 BWL  Betriebswirtschaftslehre
2                  MB              Maschinenbau
3                 INF                Informatik
